# Group 1 - Project: Predicting Loan Default

Team members: Luca Matteucci, Santiago Mazzei, Srithijaa Sankepally, and Victor Floriano


## 1 - Business problem definition and data description


---



1.0 - **Problem Statement:**

Our goal is to predict what customers are more likely to default on their loan payments. By analyzing the Lending Club Loan dataset, we aim to understand the factors that contribute to loan defaults and late payments, as well as why some borrowers can't repay their loans on time, and to find out what helps borrowers succeed. We want to make the lending process better, reduce default risk, and increase profitability for lenders.

1.1 - **Data Source:**

We will use the Lending Club Loan dataset, which includes complete loan data for loans issued from 2007 to 2015. The Lending Club is a peer-to-peer lending company that matches people looking to invest money with people looking to borrow money. The dataset provides information about borrowers, loan characteristics, and loan performance. The data is freely available on Kaggle at: https://www.kaggle.com/datasets/adarshsng/lending-club-loan-data-csv




1.2 - **Data Description:**

The dataset contains approximately 2,260,668 observations and 145 variables (columns). The variables include information, such as borrower characteristics (e.g., credit scores, income, employment details), loan characteristics (e.g., loan amount, interest rate, purpose), and loan performance data (e.g., current loan status, delinquency history). Some of the variables we expect to be useful for our analysis include: `annual_inc`, `loan_amnt`, `int_rate`, `delinq_2yrs`, `purpose`, among others.

The dataset contains multiple data types, including float64 (105 instances), int64 (4 instances), and object (36 instances). Additionally, it includes several variables that necessitate preprocessing, such as:
1. Binary values assigned to 'Y'/'N'. (i.e. `hardship_flag`)
2. Object columns would work better as datetime. (i.e. `settlement_date`)
3. Variables with text input. (i.e. `desc`)

##2 - Import libraries, load the data, and sample from the original dataset


---
To avoid issues in our colab notebook, we decided to use a sample of the original data for train/test our models. Before moving to the next steps, make sure that the entire original dataset was loaded in the colab environment, the cell below with `loan_df.info()` should return DataFrame with 2,260,667 records.


In [13]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [2]:
#Create drive and load the original dataset
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/BU_MSBA/BA810/Data/loan.csv')

Mounted at /content/drive


<ipython-input-2-da8de02a8886>:4: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/BU_MSBA/BA810/Data/loan.csv')


In [3]:
#Display the number of entries, columns, and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Columns: 145 entries, id to settlement_term
dtypes: float64(105), int64(4), object(36)
memory usage: 2.4+ GB


Our target variable contains multiple values, we will transform this column into a binary 0/1 variable. Additionally, records with `Current` outcomes are essentially loans for which we do not have labels yet. Therefore, we decided to remove those from our analysis.

In [4]:
#Check distribution of target varible in the original dataset
print('Distribution of loan_status values in the original dataset:\n')
df['loan_status'].value_counts(normalize=True) * 100

Distribution of loan_status values in the original dataset:



Fully Paid                                             46.090448
Current                                                40.682444
Charged Off                                            11.574234
Late (31-120 days)                                      0.968608
In Grace Period                                         0.395989
Late (16-30 days)                                       0.165305
Does not meet the credit policy. Status:Fully Paid      0.087939
Does not meet the credit policy. Status:Charged Off     0.033663
Default                                                 0.001371
Name: loan_status, dtype: float64

In [5]:
#Dropping records were the status of our target variable (loan_status)
#is Current

#Get index value for all rows where the loan_status = Current
row_indexes_drop = df[df['loan_status'] == 'Current'].index

#Drop selected rows
df.drop(row_indexes_drop, axis=0, inplace=True)

#Check if the Current values were dropped
df['loan_status'].value_counts(normalize=True) *100

Fully Paid                                             77.701192
Charged Off                                            19.512324
Late (31-120 days)                                      1.632919
In Grace Period                                         0.667575
Late (16-30 days)                                       0.278678
Does not meet the credit policy. Status:Fully Paid      0.148251
Does not meet the credit policy. Status:Charged Off     0.056750
Default                                                 0.002312
Name: loan_status, dtype: float64

To create a representative sample from our original dataset, we used the `train_test_split` method to create a stratified sample.

In [6]:
#Split the data into a left_out portion and our stratified sample
left_out, sample_df = train_test_split(df, test_size=0.1,
                                       stratify=df['loan_status'],
                                       random_state=42)

sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134098 entries, 806938 to 1333146
Columns: 145 entries, id to settlement_term
dtypes: float64(105), int64(4), object(36)
memory usage: 149.4+ MB


After creating our sample we check the distribution of loan_status to verify if it resembled the original dataset. While there was small difference in the distribution of loan_status values, the distribution is very similar to the original dataset.

In [7]:
#Check loan_status distribution for sample_df
print('Distribution of loan_status values in the sample dataset:\n')
sample_df['loan_status'].value_counts(normalize=True) * 100

Distribution of loan_status values in the sample dataset:



Fully Paid                                             77.701383
Charged Off                                            19.511850
Late (31-120 days)                                      1.633134
In Grace Period                                         0.667422
Late (16-30 days)                                       0.278901
Does not meet the credit policy. Status:Fully Paid      0.148399
Does not meet the credit policy. Status:Charged Off     0.056675
Default                                                 0.002237
Name: loan_status, dtype: float64

To clean RAM space in our Colab environment, we proceeded to remove the original dataset and the unused data

In [8]:
#Remove original dataset and unused data
del df
del left_out
del row_indexes_drop

#Manually trigger garbage collection
gc.collect()

0

##3 - ADD OUR INITIAL DATA EXPLORATION HERE?


---



## 4 - Prepare Data Processing for Machine Learning


---


###4.1 - Drop columns with too many missing values

In our set on predictors, one issue was that a number of features had an excessive number of missing values, we decided to drop those columns.

1.  Among the various columns, `id` and `member_id` serve as unique identifiers. The `id` column is distinct for each loan record, while `member_id` uniquely identifies each borrower. Besides the extensive number of unique values, most data from these features was missing, so we dropped both columns.

2. For the remaining variables in our dataset, we adopted a threshold-based approach, deciding to eliminate any predictors where over 35% of the data was missing. Given the significant lack of available information for those predictors, imputations methods would yield untrustworthy results.
  * 58 columns had more than 35% of their data missing and were dropped from our analysis.

In [9]:
#Create a Series with the % of missing values for each feature
missing_percent = sample_df.isnull().mean() * 100

#Select only columns with more than 35% of their data missing
#and extract the names of columns to be dropped
columns_to_drop = missing_percent[missing_percent>35.0].index

#Drop columns from sample_df
sample_df.drop(columns=columns_to_drop, inplace=True)

###4.2 - Description of Remaining Columns and Summary of Further Modifications:

Based on the descriptions of the columns, it appears that columns with numeric data should be processed as numerical values, and those labeled as 'object' columns as categoric.

* `loan_status` : Our TARGET varible, the status of the loan. While multiple values appeared in the data, we re-mapped those to (0 = no default) or (1 = default) | categoric
* `loan_amnt`: The listed amount of the loan applied for by the borrower | **numeric**
* `funded_amnt` : The total amount committed to that loan at that point in time | **numeric**
* `funded_amnt_inv` : NO DESCRIPTION FOUND, values also seem very similar to funded amount | **dropped**
* `term` : Number of payments on the loan. Values are in months and can be either 36 or 60, re-mapped to '30' and '60' (originally ' 30 months' and ' 60 months') | **categoric**
* `int_rate`: The effective interest rate is equal to the interest rate on a Note reduced by Lending Club's estimate of the impact of uncollected interest prior to charge off | **numeric**
* `installment` : The monthly payment owed by the borrower if the loan originates | **numeric**
* `grade`: LC assigned loan grade, values from 'A' to 'F'. | **categoric**
* `sub_grade`: LC assigned loan subgrade (i.e. 'A5')| **categoric**
* `emp_title`: The job title supplied by the Borrower when applying for the loan, too many unique values | **dropped**
* `emp_length`: Employment length in years. Possible values are between 0 and 10. 0 means less than one year and 10 means ten or more years. | **categoric**
* `home_ownership`:  The home ownership status provided by the borrower during registration | **categoric**
* `annual_inc`: The self-reported annual income provided by the borrower during registration | **numeric**
* `verification_status` : Income was verified by LC, not verified, or if the income source was verified | **categoric**
* `issue_d` : Date value in which the loan was issued. Changed to datetime and then converted into numeric value represnting the time since the earliest day in the dataset | **numeric**
* `pymnt_plan` : Payment plan 'y' or 'n' | **categoric**
* `purpose`: A category provided by the borrower for the loan request(14 unique categories) | **categoric**
* `title`: Loan title provided by the borrower, a text based varible (+8k unique entries), since NLP is outside of the score of this project we dropped this feature | **dropped**
* `zip_code`
* `addr_state`: The state provided by the borrower in the loan application | **categoric**
* `dti`:  Ratio calculated of borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income. | **numeric**
* `delinq_2yrs`: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years | **numeric**
* `earliesst_cr_line`: The date the borrower's earliest reported credit line was opened
* `inq_last_6mths`: The number of inquiries in the past 6 months.| **numeric**
* `open_acc`: The number of open credit lines.| **numeric**
* `pub_rec`: The number of derogatory public records.| **numeric**
* `revol_bal`: The total credit revolving balance.| **numeric**
* `revol_util`: The revolving line utilization rate.| **numeric**
* `total_acc`: The total number of credit lines.| **numeric**
* `initial_list_status`: The initial listing status of the loan. It has discrete values (W, F). Convert it to a categorical variable. | **categorical**
* `out_prncp`: The remaining outstanding principal for the total amount funded.| **numeric**
* `out_prncp_inv`: The remaining outstanding principal funded by investors.| **numeric**
* `total_pymnt`: The payments received to date for the total amount funded.| **numeric**
* `total_pymnt_inv`: The payments received to date for the portion funded by investors.| **numeric**
* `total_rec_prncp`: The principal received to date.| **numeric**
* `total_rec_int`: The interest received to date.| **numeric**
* `total_rec_late_fee`: The late fees received to date.| **numeric**
* `recoveries`: The post-charge-off gross recovery.| **numeric**
* `collection_recovery_fee`: The post-charge-off collection fee.| **numeric**
* `last_pymnt_d`: The last month payment was received. Already an object type.| **object**
* `last_pymnt_amnt`: The last total payment amount received.| **numeric**
* `last_credit_pull_d`: The most recent month credit was pulled. Convert it to a categorical variable.| **categorical**
* `collections_12_mths_ex_med`: The number of collections in the last 12 months excluding medical collections.| **numeric**
* `policy_code`: It has only two possible values, and it's not clear what do those values mean. It might not provide meaningful information.| **drop**
* `application_type`: Indicates whether the loan is an individual or joint application. Already object type.| **object**
* `acc_now_delinq`: The number of accounts on which the borrower is now delinquent. The values for this variable are 0, 1, 2, 3, 5, 14. | **numerical**
* `tot_coll_amt`: Total collection amounts ever owed | **numerical**
* `tot_cur_bal`: Total current balance of all accountsm | **numerical**
* `total_rev_hi_lim`: Total revolving (perpetual like credit cards) high credit limit | **numerical**
* `acc_open_past_24mths`: Number of account opened in past 24 months. | **numerical**
* `avg_cur_bal`: Average current balance of all accounts |**numerical**
* `bc_open_to_buy`: Total open to buy on revolving bankcards (amount open). | **numerical**
* `bc_util`: Ratio of total current balance to credit limit for all bankcard accounts. | **numerical**
* `chargeoff_within_12_mths`: Number of charge-offs (considered a loss by the lender but still obbligation) within 12 months |**numerical**
* `delinq_amnt`: The past-due amount owed for the accounts on which the borrower is now delinquent | **numerical**
* `mo_sin_old_il_acct`: (6% NULL) Months since oldest bank installment account opened | **numerical**
* `mo_sin_old_rev_tl_op`: Months since oldest revolving account opened | **numerical**  
* `mo_sin_rcnt_rev_tl_op`:Months since most recent revolving account opened | **numerical**
* `mo_sin_rcnt_tl`: Months since most recent account opened | **numerical**
* `mort_acc`: Number of mortgage accounts. | **numerical**
* `mths_since_recent_bc`: Months since most recent bankcard account opened. | **numerical**
* `mths_since_recent_inq`:(13% NULL) SHOULD BE THE REQUEST NOT NECESSARILY OPEN USUALLY > OF THE OTHERS VARIABLES Months since most recent inquiry. | **numerical**
* `num_accts_ever_120_pd`: Number of accounts over 120 or more days past due | **numerical**
* `num_actv_bc_tl`: Number of currently active bankcard accounts | **numerical**
* `num_actv_rev_tl`: Number of currently active revolving account | **numerical**
* `num_bc_sats`: Number of satisfactory (current or being paid) bankcard accounts | **numerical**
* `num_bc_tl`: Number of bankcard accounts | **numerical**
* `num_il_tl` : Number of installment accounts | **numeric**
* `num_op_rev_tl` : Number of open revolving accounts | **numeric**
* `num_rev_accts` : Number of revolving accounts | **numeric**
* `num_sats` : Number of satisfactory accounts | **numeric**
* `num_tl_120dpd_2m` : Number of accounts currently 120 days past due (updated in past 2 months) | **numeric**
* `num_tl_30dpd` : Number of accounts currently 30 days past due (updated in past 2 months) | **numeric**
* `num_tl_90g_dpd_24m` : Number of accounts 90 or more days past due in last 24 months | **numeric**
* `num_tl_op_past_12m` : Number of accounts opened in past 12 months | **numeric**
* `pct_tl_nvr_dlq` : Percent of trades never delinquent | **numeric**
* `percent_bc_gt_75` : Percentage of all bankcard accounts > 75% of limit | **numeric**
* `pub_rec_bankruptcies` : Number of public record bankruptcies | **numeric**
* `tax_liens` : Number of tax liens | **numeric**
* `tot_hi_cred_lim` : Total high credit/credit limit | **numeric**
* `total_bal_ex_mort` : Total credit balance excluding mortgage | **numeric**
* `total_bc_limit` : Total bankcard high credit/credit limit | **numeric**
* `total_il_high_credit_limit` : Total installment high credit/credit limit | **numeric**
* `hardship_flag` : Flags whether or not the borrower is on a hardship plan | **categorical**
* `disbursement_method` : The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY | **categorical**
* `debt_settlement_flag` : Flags whether or not the borrower, who has charged-off, is working with a debt-settlement company | **categorical**




In [10]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134098 entries, 806938 to 1333146
Data columns (total 87 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   134098 non-null  int64  
 1   funded_amnt                 134098 non-null  int64  
 2   funded_amnt_inv             134098 non-null  float64
 3   term                        134098 non-null  object 
 4   int_rate                    134098 non-null  float64
 5   installment                 134098 non-null  float64
 6   grade                       134098 non-null  object 
 7   sub_grade                   134098 non-null  object 
 8   emp_title                   125570 non-null  object 
 9   emp_length                  126333 non-null  object 
 10  home_ownership              134098 non-null  object 
 11  annual_inc                  134098 non-null  float64
 12  verification_status         134098 non-null  object 
 13  issue_d 

In [11]:
#Check for unique values
sample_df['term'].unique()

#Re-mapping values
sample_df['term'].replace({' 36 months':'36', ' 60 months':'60'}, inplace=True)

In [12]:
#When checking the possible values for emp_title we
#found that in our sample of around 200k records, there
#are more than 80k unique values, this would create problems when
#splitting the data and could over complicate our models, so
#we decided to drop this predictor.
print(sample_df['emp_title'].value_counts())

#Drop predictor
sample_df.drop(columns=['emp_title'], inplace=True)

Teacher                    2091
Manager                    1964
Owner                      1033
Registered Nurse            895
RN                          838
                           ... 
Internal Audit Director       1
Heartland Health Care         1
SR SYSTEM ANALYSTS            1
Staff Product Engineer        1
Forensic Scientist            1
Name: emp_title, Length: 56903, dtype: int64


In [24]:
#issue_d is the date the loan was funded.
#Too many unique values, it would slow down our models too much. However, since
#there is an ordinal component to the dates we decided to convert them into
#numeric
print(sample_df['issue_d'].unique())

#Convert the columns into datetime dtype
sample_df['issue_d'] = pd.to_datetime(sample_df['issue_d'], format='%b-%Y')

#Select the earliest date
earliest_issue_d = sample_df['issue_d'].min()

#Change values to represent the days since the earliest date on record
sample_df['issue_d'] = (sample_df['issue_d'] - earliest_issue_d).dt.days


In [25]:
#Loan status has multiple different values.
#For simplicity re-map to either default or no default
print(sample_df['loan_status'].unique())


sample_df['loan_status'].replace({ 'Fully Paid':0,
                                  'Charged Off':1, 'Late (31-120 days)':1,
                                  'In Grace Period':0, 'Does not meet the credit policy. Status:Fully Paid':0,
                                  'Late (16-30 days)':1, 'Does not meet the credit policy. Status:Charged Off':1,
                                  'Default':1
}, inplace=True)

['Charged Off' 'Fully Paid' 'In Grace Period' 'Late (31-120 days)'
 'Does not meet the credit policy. Status:Fully Paid' 'Late (16-30 days)'
 'Does not meet the credit policy. Status:Charged Off' 'Default']


In [26]:
sample_df['loan_status'].value_counts()

0    105290
1     28808
Name: loan_status, dtype: int64

In [27]:
#Check title feature
#title seems to contain explanations that the
#borrower provided for the loan, or simply 'thank you'
#messages (unstructured data). Given that NLP is outside of the scope of this
#project we decided to drop this feature
print(sample_df['title'].value_counts())

sample_df.drop(columns=['title'], inplace=True)

Debt consolidation         65993
Credit card refinancing    24618
Home improvement            7423
Other                       6598
Major purchase              2339
                           ...  
Consolidation Heaven           1
pontiac grand prix             1
Apartment Bills                1
E350                           1
10-month plan                  1
Name: title, Length: 8651, dtype: int64


In [28]:
#Too many unique values - Dropped
len(sample_df['zip_code'].unique())

sample_df.drop(columns=['zip_code'], inplace=True)

In [29]:
#Too many unique values, did not seem as relevant as issue_d (DROP)
sample_df['earliest_cr_line'].unique()

sample_df.drop(columns=['earliest_cr_line'], inplace=True)

In [34]:
#Change initial_list_status to categoric
sample_df['initial_list_status'].astype('object')


806938     w
2091371    f
2002719    f
724430     w
1317626    f
          ..
909669     f
2229967    w
1334828    f
874633     w
1333146    f
Name: initial_list_status, Length: 134098, dtype: object

In [30]:
#Drop policy code variable (unavailable information on variable)

sample_df.drop(columns=['policy_code'], inplace=True)

###4.3 - Create Train/Test Split

In [ ]:
#Train/test split

X = sample_df.drop('loan_status', axis=1)
y = sample_df['loan_status'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0, stratify=y)

#Create copies to experiment on before adding to pipeline
X_train_copy = X_train.copy();
y_train_copy = y_train.copy();

###4.4 - Imputation of Null Values

After removing the features with more than 35% missing values, some predictors still contained some null values. We decided to test SimpleImputer, a method that will later be applied in our pre-processing pipeline (SECTION X)


In [ ]:
#Note: test will be done on numeric columns
#Create mean imputer
mean_imputer = SimpleImputer(strategy="mean")

#Select only numerical variables
X_train_copy_num = X_train_copy.select_dtypes(include=[np.number])

#Fit/Transform
X_train_copy_imp = mean_imputer.fit_transform(X_train_copy_num)

#Create dataframe with results
X_train_copy_imp_df = pd.DataFrame(X_train_copy_imp, columns=X_train_copy_num.columns, index=X_train_copy_num.index)

#Check for any null values remaining
print('Missing values remaining (numeric cols):', X_train_copy_imp_df.isnull().sum().sum())


Missing values remaining (numeric cols): 0


###4.4 - Feature Scaling

To prepare our data for models that require normalized values, we tested feature scaling in this section before applying it on our pre-processing pipeline

In [ ]:
#Create standard scaler object
std_scaler = StandardScaler()

#Fit/Transform
X_train_copy_num_std_scaled = std_scaler.fit_transform(X_train_copy_num)

#Create dataframe with results
X_train_copy_num_std_scaled_df = pd.DataFrame(X_train_copy_num_std_scaled, columns=X_train_copy_num.columns, index=X_train_copy_num.index)
X_train_copy_num_std_scaled_df.head(5)

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
422327,-0.984018,-0.983687,-0.981291,0.099898,-0.905076,-0.340400,0.532062,-0.355482,0.472205,0.069344,...,-0.171136,2.147643,0.651981,-1.171893,2.375307,-0.12217,-0.823800,-0.760504,-0.391661,-0.780861
1750701,-0.570255,-0.569812,-0.567596,0.255178,-0.408099,-0.101021,0.542821,0.804096,1.592586,-0.989852,...,-0.171136,-0.041263,-0.126933,0.482326,-0.353796,-0.12217,-0.495279,-0.031372,1.694158,-0.414982
515423,1.629224,1.630260,1.631517,-0.063664,0.876241,0.311592,-0.066178,-0.355482,-0.648175,1.481605,...,-0.171136,-0.588489,0.651981,-0.253801,-0.353796,-0.12217,1.373387,0.882180,0.140612,1.434579
2147172,-0.330708,-0.330201,-0.328089,-0.452898,-0.198925,-0.164016,-0.632139,-0.355482,-0.648175,-0.813319,...,-0.171136,-0.588489,0.651981,0.667048,-0.353796,-0.12217,-0.830420,-0.694128,-0.664289,-0.850416
1963121,-0.439593,-0.439115,-0.436956,-0.436335,-0.319844,-0.334100,-0.293567,-0.355482,-0.648175,-0.813319,...,-0.171136,-0.588489,0.651981,1.585139,-0.353796,-0.12217,-0.899293,-0.826759,-0.703236,-0.972786
